In [1]:
from graph_tool.all import *
import graph_tool.all as gt
import numpy as np
import random
import os
from tqdm import tqdm

In [2]:
def create_network(n, m, beta, seed=None):
    if seed is not None:
        np.random.seed(seed=seed)

    initial_nodes = m + 1
    g = gt.complete_graph(initial_nodes, directed=False)
    node_list = list(range(initial_nodes))
    node_id_property = g.new_vertex_property("int")

    for i in range(initial_nodes):
        node_id_property[g.vertex(i)] = i

    node_id = initial_nodes - 1

    # 初期ネットワークの次数リストを作成
    K_array_before = [v.out_degree() for v in g.vertices()]

    for i in range(initial_nodes, n):

        if beta == 0 or beta >= 1:
            K_array = np.array(K_array_before) ** beta
        elif beta == 1 / 2 or beta == 0.7:
            K_array = np.sqrt(np.array(K_array_before))
        else:
            ipa = abs(beta)
            K_array = 1 / np.power(np.array(K_array_before), ipa, dtype=np.float64)

        g.add_vertex()
        K_array_before.append(0)  # 新しく追加したノードの次数を0で初期化

        for _ in range(m):
            # 次数に比例した確率ベクトルを生成
            s = np.sum(K_array)
            parray = K_array / s
            parray /= np.sum(parray)  # 確率の合計を1.0に調整

            # 次数に比例した確率で既存のノードを選び、エッジとする
            cumulative_probs = np.cumsum(parray)
            r = random.random()
            new = node_list[np.searchsorted(cumulative_probs, r)]

            # エッジを追加
            g.add_edge(g.vertex(i), g.vertex(new))

            K_array_before[i] += 1
            K_array_before[new] += 1

            # ノードの次数を0に設定（選ばれたノードは次回選ばれないように）
            K_array[node_list.index(new)] = 0.

        node_list.append(i)
        node_id += 1
        node_id_property[g.vertex(i)] = node_id

    g.vertex_properties["node_id"] = node_id_property
    return g

In [3]:
#チェーン構造を解消するためのリワイヤリングコード
def rewire_network(g, rewiring_times):
    #print("random_rewire開始")
    gt.random_rewire(g, model = "configuration", n_iter = rewiring_times, edge_sweep=True)
    #print("random_rewire終了")
    return g

In [4]:
# Define function to check sum_u
def check_sum_u(g, target_sum):
    l = gt.label_largest_component(g)
    u = gt.GraphView(g, vfilt=l)
    sum_u = u.num_vertices()
    return sum_u == target_sum

In [5]:
Node = 50000

m = 2

times = 50

#beta_list = [1, 1/2, 0,-1,-3,-5,-10,-20,-50,-100]
beta_list =[0.7]

In [6]:
for beta in beta_list:
    directory = os.path.expanduser(f"~/o_t_hayashilab/network_data/graph-tool/pure_pk/N={Node}/m={m}/beta={beta}")
    os.makedirs(directory, exist_ok=True)

    for i in tqdm(range(times)):
        while True:
            g = create_network(Node, m, beta, seed=None)
            g = rewire_network(g, Node)
            if check_sum_u(g, Node):
                filename = os.path.join(directory, f"{i}.gt.gz")
                g.save(filename)
                break
            else:
                print("ランダム化がうまくいきませんでした")

100%|██████████| 50/50 [42:56:28<00:00, 3091.77s/it]   


In [ ]:
for beta in beta_list:
    for i in range(times):
        g = load_graph(f"~/o_t_hayashilab/Network_data/graph-tool/test_pk/N={Node}/m={m}/beta={beta}/{i}.gt.gz")
        total_loop = 0
        multiple_edge = 0
        
        # グラフ g のすべてのノードとエッジを調査
        for v in tqdm(g.vertices()):
            # 自己ループの検出
            if g.edge(v, v) is not None:
                print(f"ノード {int(g.vp.node_id[v])} に自己ループが存在します。")
                total_loop += 1

            # 多重リンクの検出
            for u in g.vertices():
                if u != v:
                    edges = list(g.edge(v, u, all_edges=True))
                    if len(edges) > 1:
                        print(f"ノード {int(g.vp.node_id[v])} と {int(g.vp.node_id[u])} の間に多重リンクが存在します。")
                        multiple_edge += 1

        print(f"自己ループは{int(total_loop)}個あります。")
        print(f"多重リンクは{int(multiple_edge)}個あります。")